<a href="https://colab.research.google.com/github/atejap05/pos_data_science_furg/blob/main/disciplinas/Machine_Learning_I/titanic/Furg_ECD_05_Machine_Learning_I_Tarefa_Naufr%C3%A1gio_do_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Especialização em Ciência de Dados - FURG
## Machine Learning I - Tarefa: Naufrágio do Titanic
### Prof. Marcelo Malheiros

Código adaptado de Aurélien Geron (licença Apache-2.0)

---

Esta tarefa é para você **observar** e **analisar** este processo de Machine Learning.

Adicionalmente, sugere-se que você também experimente com os dados e com os algoritmos, fazendo algumas das modificações indicadas em várias partes deste _notebook_.

Note que não é preciso escrever mais código, apenas modificar o código já fornecido.

Um questionário _online_ dentro da disciplina no AVA será disponibilizado para coletar sua análise. Este questionário será também uma das tarefas avaliativas desta disciplina.

# Problema

O problema aqui descrito é uma **tarefa de classificação**. Com base nos dados reais dos passageiros do navio RMS Titanic, precisamos ajustar um **modelo de previsão** para ser capaz de prever a chance de uma dada pessoa sobreviver ou não ao naufrágio (ocorrido em 15 de abril de 1912).

# Inicialização

Aqui importamos as bibliotecas fundamentais de Python para este _notebook_.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

# Conjunto de dados

Este _dataset_ já está separado em dois conjuntos, um para **treino** (`titanic_treino.csv`) e outro para **teste** (`titanic_teste.csv`). 

Os conjuntos de treino e de teste já estão separados. Basta treinar um **modelo** com o conjunto de treino e analisar uma **medida de desempenho** sobre este.

Quando esta medida indicar que o modelo tem boa qualidade, então deve-se usar o conjunto de teste para gerar uma nova previsão, comparar esta com o resultado já conhecido para cada passageiro.

**Não se esqueça de fazer o upload dos dados para o Colaboratory, antes de rodar a célula a seguir.**


In [ ]:
# leitura dos conjuntos de dados

treino = pd.read_csv('titanic_treino.csv')
treino.set_index('id', inplace=True)

teste = pd.read_csv("titanic_teste.csv")
teste.set_index('id', inplace=True)

Cada conjunto de dados foi colocado em um `DataFrame` (da biblioteca Pandas).

Vamos visualizar a seguir os primeiro cinco registros da base de treino:

In [ ]:
treino.head(5)

Os dois conjuntos têm a mesma estrutura de atributos, onde cada linha representa um passageiro. As colunas são os atributos, definidos da seguinte forma:

- **id**: um número sequencial, único para cada passageiro

- **sobrevivente**: indica se o passageiro sobreviveu (1) ou não (0) ao naufrágio

- **classe**: qual a classe da passagem comprada, da primeira (1) até a terceira (3) classe

- **nome**: nome completo do passageiro (incluindo apelido, se for o caso)

- **sexo**: sexo masculino (M) ou feminino (F)

- **idade**: idade em anos

- **familia1**: número de cônjuges, de irmãos e de irmãs presentes no navio

- **familia2**: número de pais e de filhos do passageiro, também presentes no navio

- **tiquete**: código do tíquete de embarque

- **tarifa**: preço pago pela passagem, em libras esterlinas

- **cabine**: código da cabine do passageiro

- **embarque**: porto de embarque do passageiro, sendo Cherbourg (C), Queenstown (Q) ou  Southampton (S)

# Análise dos dados

Aqui vamos computar algumas estatísticas sobre a base original de dados. Estas estatísticas são importantes tanto para perceber quais operações de preprocessamento serão necessárias como para escolher quais _features_ vamos usar no treinamento.

Uma função muito útil para isso é a `.info()`, que mostra o tipo de dados e o número de valores presentes em cada coluna:

In [ ]:
treino.info()

Para o conjunto de treino há um total de 891 registros.

É possível perceber que há valores faltantes nas colunas `idade`, `cabine` e `embarque`. Na etapa de preprocessamento precisamos completar estes valores (por exemplo, preenchendo as idades faltantes com a mediana de todas as idades). Ou pode-se simplesmente descartar algumas colunas, como faremos depois com `cabine`.

Podemos também olhar estatísticas sobre os valores das colunas numéricas usando `.describe()':

In [ ]:
treino.describe()

Vale a pena também analisar as colunas que contém poucos valores, chamadas de categóricas, para identificar as classes presentes. Faremos isso chamando a função `.value_counts()` em cada uma.

In [ ]:
treino['sobrevivente'].value_counts()

In [ ]:
treino['classe'].value_counts()

In [ ]:
treino['sexo'].value_counts()

In [ ]:
treino['embarque'].value_counts()

Como o atributo que desejamos prever, `sobrevivente` só tem dois valores, temos uma tarefa de **classificação binária**. Isso afeta diretamente quais algoritmos podemos usar para fazer o treino, posteriormente.

Além disso, de 891 passageiros do conjunto de treino apenas 342 sobreviveram, o que equivale a 38%. O uso da **acurácia** como medida de desempenho parece adequado neste caso, uma vez que não há uma grande disparidade na quantidade de valores 0 e de valores 1.

# Correlação entre atributos

Apenas para gerar uma intuição sobre os dados, vamos visualizar a correlação entre os atributos do conjunto de treino.

Para isso usamos a função `.corr()` para calcular o coeficiente de correlação (também chamado de R de Pearson) entre cada par de atributos de um DataFrame.

Então visualizamos a correlação com o nosso atributo alvo, `sobrevivente`:

In [ ]:
# cálculo da matriz de correlação
corr = treino.corr()

# quanto cada atributo se correlaciona com o valor de 'sobrevivente'
corr['sobrevivente']

Note que é possível visualizar a tabela inteira, que cruza cada os atributos com todos os demais, como visualizado abaixo. A escala de cores é de um azul mais intenso para valores negativos (no mínimo -1) e de vermelhos mais intensos para valores positivos (no máximo 1). Valores próximos ao zero são também mais próximos do branco.

Ainda que haja correlação forte entre alguns atributos, para esta análise só interessam correlações envolvendo o atributo `sobrevivente`, ou seja, da linha superior ou da coluna mais à esquerda da tabela.

In [ ]:
corr.style.background_gradient(axis=None, cmap='bwr').set_precision(2)

# Separação dos dados

Agora vamos separar cada conjunto de dados em duas partes: `features`, que são os atributos sobre os quais treinaremos o modelo, e `rotulos`, que contém o atributo a ser previsto (no caso, apenas `sobrevivente`).
        
Para simplificar, vamos ignorar os atributos `nome`, `tiquete` e `cabine`, que são meramente textuais.

In [ ]:
# separacão das features
treino_features = treino[['classe', 'sexo', 'idade', 'familia1', 'familia2', 'tarifa', 'embarque']]
teste_features  =  teste[['classe', 'sexo', 'idade', 'familia1', 'familia2', 'tarifa', 'embarque']]

# separação dos rótulos
treino_rotulos = treino[['sobrevivente']]
teste_rotulos  =  teste[['sobrevivente']]

Vamos ver o número de linhas e de colunas de cada subconjunto criado:

In [ ]:
print('treino_features:', treino_features.shape)
print('treino_rotulos: ', treino_rotulos.shape)
print('teste_features: ', teste_features.shape)
print('teste_rotulos:  ', teste_rotulos.shape)

# Preprocessamento

Aqui faremos a etapa de preprocessamento, necessária para transformar os dados brutos em valores mais adequados para os algoritmos de Machine Learning.

Lembrando, todo o processo de transformação dos dados é chamado de _pipeline_.

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# aqui definimos uma etapa auxiliar do pipeline para atributos categóricos:
# esta substitui valores faltando pelo mais frequente de cada coluna
# (não se preocupe com os detalhes do código)

from sklearn.base import BaseEstimator, TransformerMixin

class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X], index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

Atributos numéricos (contínuos) e atributos categóricos (discretos) precisam ser processados separadamente.

O trecho a seguir define um _pipeline_ genérico, apenas para os atributos numéricos. A etapa usando `SimpleImputer(strategy='median')` preenche valores faltando com a mediana dos valores daquele atributo.

**Ajuste:** A etapa de escalonamento usando `StandardScaler()` está desativada. Depois de medir o desempenho do modelo você pode ativar esta etapa, rodando novamente o preprocessamento e o treino, para verificar se melhora a qualidade do modelo.

In [ ]:
# pipeline para atributos numéricos

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    #('std_scaler', StandardScaler()),
])

O trecho a seguir define um pipeline genérico, apenas para os atributos categóricos.

A etapa usando `MostFrequentImputer()` preenche valores faltando com o mais frequente dos valores daquele atributo. Já a etapa seguinte, usando `OneHotEncoder(sparse=False)` expande cada atributo que não é numérico (`classe`, `sexo` ou `embarque`) para um conjunto de atributos binários. Esta precisa ser a última etapa desse _pipeline_.

In [ ]:
# pipeline para atributos categóricos

from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
    ('imputer', MostFrequentImputer()),
    ('encoder', OneHotEncoder(sparse=False)),
])

A seguir usamos o recurso `ColumnTransformer`, muito útil, para aplicar determinados _pipelines_ para apenas alguns atributos dos dados. Assim temos um _pipeline_ inteligente que processa os dados de uma só vez, mas executa operações diferentes conforme a natureza de cada atributo.

In [ ]:
# pipeline combinando atributos numéricos com atributos categóricos

from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, ['idade', 'familia1', 'familia2', 'tarifa']),
    ('cat', cat_pipeline, ['classe', 'sexo', 'embarque']),
])

Enfim, processamos os features de treino, gerando um novo conjunto de dados pronto para o treinamento.

In [ ]:
X_treino = full_pipeline.fit_transform(treino_features)
print('X_treino:', X_treino.shape)
print(X_treino)

Aqui vamos extrair somente os dados brutos dos rótulos, e colocar em `y_treino`, que é o formato esperado pelos algoritmos de Machine Learning.

In [ ]:
y_treino = treino_rotulos.values.ravel()
print('y_treino:', y_treino.shape)
#print(y_treino)

# Seleção do modelo

Aqui selecionamos um modelo para o aprendizado, baseado em um dos possíveis algoritmos de classificação binária. 

**Ajuste:** Depois de medir o desempenho do modelo você pode habilitar outros algoritmos. Note que apenas um par de comandos deve estar ativo (a linha `from` e a correspondente criação do `classificador`).

- Para o algoritmo `SVC` você pode mudar o parâmetro para `gamma='auto'`.

- Para o algoritmo `SGDClassifier` você pode mudar o parâmetro para `loss='log'`.

- Para o algoritmo `LogisticRegression` você **precisa ter os dados escalonados** como parte do _pipeline_ numérico. Ou seja, é preciso que a linha do `StandardScaler()` esteja habilitada e que os dados sejam novamente preprocessados.

- Para o algoritmo `RandomForestClassifier` você pode aumentar o parâmetro `n_estimators`, por exemplo para 10 ou 100.

In [ ]:
from sklearn.svm import SVC
classificador = SVC(random_state=42, gamma='scale')

#from sklearn.linear_model import SGDClassifier
#classificador = SGDClassifier(random_state=42, loss='hinge')

#from sklearn.linear_model import LogisticRegression
#classificador = LogisticRegression(random_state=42)

#from sklearn.ensemble import RandomForestClassifier
#classificador = RandomForestClassifier(random_state=42, n_estimators=1)

# Treino e medida de desempenho

Vamos treinar e avaliar um classificador fazendo a **validação cruzada** do conjunto de treino. O parâmetro `cv` indica o número de dobras (ou _folds_), que é o número de vezes em que o conjunto é repartido, treinado e mensurado.

Como medida de desempenho vamos usar a **acurácia** (_accuracy_), que indica o percentual de acertos na previsão.

Como a validação cruzada devolve várias medidas, guardadas na lista `scores`, calculamos e exibimos uma média aritmética simples dessas medidas.

In [ ]:
# validação cruzada

from sklearn.model_selection import cross_val_score

scores = cross_val_score(classificador, X_treino, y_treino, cv=10, scoring='accuracy')
print('{:.2%}'.format(scores.mean()))

**Opcional:** Para melhorar o modelo existem diversas possibilidades de **ajuste** do modelo:

- A ação mais direta é simplesmente testar outros algoritmos, como mostrado na seção "Seleção do Modelo".

- Você também pode ativar o escalonamento de _features_ na seção "Preprocessamento" e ver se houve ganho.

- Outra possibilidade é reduzir o número de _features_, simplesmente retirando atributos numéricos e categóricos do _pipeline_ `full_pipeline` na seção "Preprocessamento".

**Avançado**: Ainda é possível criar novas _features_ (o que precisa ser feito na seção "Separação dos dados"). Seguem três sugestões (que precisariam ser repetidas para o conjunto `teste`):

    # novo atributo (categórico) agrupando idades por faixas
    treino['faixa_etaria'] = treino['idade'] // 15 * 15
    treino[['faixa_etaria', 'sobrevivente']].groupby(['faixa_etaria']).mean()    

    # novo atributo (numérico ou categórico), agrupando dados sobre familiares
    treino['total_parentes'] = treino['familia1'] + treino['familia2']
    treino[['total_parentes', 'sobrevivente']].groupby(['total_parentes']).mean()

    # novo atributo (binário e categórico) identificando passageiros viajando sozinhos
    # (apenas 30% destes sobreviveram)
    treino['sozinho'] = (treino['familia1'] + treino['familia2']) == 0

# Teste final do modelo

Se você estiver satisfeito com a qualidade do modelo obtido (com base apenas nos rótulos de treino), então podemos fazer o teste final do modelo.

Vamos gerar a versão treinada do modelo, preprocessar os dados de teste e então gerar previsões.

Note que o treinamento do modelo é feito de fato aqui, com a chamada de `.fit()`. A validação cruzada feita anteriormente gera modelos temporários para as medidas e os descarta, mas não deixa o modelo treinado.

In [ ]:
# treinamento
classificador.fit(X_treino, y_treino)

# preprocessamento dos dados de teste
X_teste = full_pipeline.transform(teste_features)

# geração das previsões
y_teste_pred = classificador.predict(X_teste)

Agora vamos avaliar a acurácia novamente, porém comparando o valores previstos com base nos dados de teste contra os rótulos reais para os mesmos dados de teste.

In [ ]:
# rótulos reais
y_teste = teste_rotulos.values.ravel()

# medida de acurácia
from sklearn.metrics import accuracy_score
score = accuracy_score(y_teste, y_teste_pred)
print('{:.2%}'.format(score))

# Gerando novas previsões

Uma vez treinado e testado, o modelo pode ser usado para gerar previsões sobre dados nunca antes vistos.

No exemplo abaixo um `DataFrame` com o mesmo formato dos conjuntos de `features` é criado, com apenas uma linha. Para essa pessoa fictícia, podemos usar o modelo para gerar uma previsão de sua possível sobrevivência ou não ao naufrágio do Titanic:

In [ ]:
anon = pd.DataFrame(columns=['classe', 'sexo', 'idade', 'familia1', 'familia2', 'tarifa', 'embarque'],
                    data   =[[      2,    'M',     40,           1,          0,     50.0,        'Q']])

In [ ]:
X_anon = full_pipeline.transform(anon)
y_anon = classificador.predict(X_anon)
y_anon[0]